# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [4]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [5]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [6]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple, Inc.',
 'primaryExchange': 'NSAADQ',
 'calculationPrice': 'tops',
 'open': None,
 'openTime': None,
 'openSource': 'afoiflci',
 'close': None,
 'closeTime': None,
 'closeSource': 'ficiflao',
 'high': 475.98,
 'highTime': 1609223220228,
 'highSource': 'tr pm lareIXeice Ei',
 'low': 470.74,
 'lowTime': 1628999218300,
 'lowSource': 'pt5d e lne ydeic1mreuai',
 'latestPrice': 485.52,
 'latestSource': 'IEX real time price',
 'latestTime': '1:05:29 PM',
 'latestUpdate': 1610695016484,
 'latestVolume': 29586645,
 'iexRealtimePrice': 472.42,
 'iexRealtimeSize': 101,
 'iexLastUpdated': 1670142826260,
 'delayedPrice': None,
 'delayedPriceTime': None,
 'oddLotDelayedPrice': None,
 'oddLotDelayedPriceTime': None,
 'extendedPrice': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPriceTime': None,
 'previousClose': 457.14,
 'previousVolume': 41699414,
 'change': 11.57,
 'changePercent': 0.02502,
 'volume': 30439519,
 'iexMarketPercen

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [7]:
pe_ratio = data['peRatio']
pe_ratio

35.5

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [8]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [9]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,103.130,45.55,N/A
1,AAL,13.558,-1.65,N/A
2,AAP,158.660,29.38,N/A
3,AAPL,470.800,35.9,N/A
4,ABBV,99.290,20.14,N/A
...,...,...,...,...
500,YUM,96.390,27.86,N/A
501,ZBH,141.150,712.02,N/A
502,ZBRA,286.940,31.3,N/A
503,ZION,35.410,13.36,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [10]:
final_dataframe.sort_values('Price-to-Earnings Ratio', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,RTX,65.350,2373.14,N/A
1,FIS,150.020,1524.95,N/A
2,GPC,98.302,937.83,N/A
3,ZBH,141.150,712.02,N/A
4,STZ,183.290,621.41,N/A
5,HLT,87.166,595.55,N/A
6,HRB,15.600,560.57,N/A
7,GLW,34.213,257.54,N/A
8,IQV,167.400,252.19,N/A
9,KMI,14.582,209.07,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [11]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [12]:
portfolio_input()

Enter the value of your portfolio:1000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [13]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,RTX,65.350,2373.14,300
1,FIS,150.020,1524.95,130
2,GPC,98.302,937.83,199
3,ZBH,141.150,712.02,138
4,STZ,183.290,621.41,106
5,HLT,87.166,595.55,224
6,HRB,15.600,560.57,1256
7,GLW,34.213,257.54,573
8,IQV,167.400,252.19,117
9,KMI,14.582,209.07,1344


## Building a Better (and More Realistic) Momentum Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [14]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['enterpriseValue']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [24]:
rv_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'Price-to-Earnings Ratio',
                'PE Percentile',
                'Price-to-Book Ratio',
                'PB Percentile',
                'EV/EBITDA',
                'EV/EBITDA Percentile',
                'EV/GP',
                'EV/GP Percentile',
                'RV Score'
                ]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        
        pe_ratio = data[symbol]['quote']['peRatio']
        pb_ratio = data[symbol]['advanced-stats']['priceToBook']
        ps_ratio = data[symbol]['advanced-stats']['priceToSales']
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['enterpriseValue']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
            
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
                                        pd.Series([
                                                    symbol, 
                                                    data[symbol]['quote']['latestPrice'], 
                                                    'N/A', 
                                                    pe_ratio,
                                                    'N/A',
                                                    pb_ratio,
                                                    'N/A',
                                                    ev_to_ebitda,
                                                    'N/A',
                                                    ev_to_gross_profit,
                                                    'N/A',
                                                    'N/A'
                                                   ], 
                                                  index = rv_columns), 
                                        ignore_index = True)
        

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBIT Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,103.040,N/A,46.9,N/A,6.519299,N/A,1.0,N/A,11.060905,N/A,N/A
1,AAL,14.055,N/A,-1.73,N/A,-60.305424,N/A,1.0,N/A,3.189168,N/A,N/A
2,AAP,162.790,N/A,28.77,N/A,3.125557,N/A,1.0,N/A,3.261244,N/A,N/A
3,AAPL,478.670,N/A,35.37,N/A,21.734212,N/A,1.0,N/A,20.615831,N/A,N/A
4,ABBV,95.610,N/A,20.13,N/A,-20.630960,N/A,1.0,N/A,7.624217,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,94.770,N/A,27.63,N/A,-3.509246,N/A,1.0,N/A,14.023448,N/A,N/A
501,ZBH,142.230,N/A,703.31,N/A,2.384249,N/A,1.0,N/A,7.081642,N/A,N/A
502,ZBRA,297.470,N/A,31.34,N/A,8.557369,N/A,1.0,N/A,8.364287,N/A,N/A
503,ZION,35.413,N/A,13.47,N/A,0.796279,N/A,1.0,N/A,NaN,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [23]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBIT,EV/EBIT Percentile,EV/GP,EV/GP Percentile,RV Score
17,AFL,37.963,N/A,9.49,N/A,0.957297,N/A,1.0,N/A,NaN,N/A,N/A
18,AIG,30.859,N/A,-5.73,N/A,0.399389,N/A,1.0,N/A,NaN,N/A,N/A
20,AIZ,129.060,N/A,20.28,N/A,1.355266,N/A,1.0,N/A,NaN,N/A,N/A
26,ALL,98.440,N/A,7.3,N/A,1.194129,N/A,1.0,N/A,NaN,N/A,N/A
39,ANTM,291.140,N/A,12.34,N/A,2.295116,N/A,1.0,N/A,NaN,N/A,N/A
40,AON,194.020,N/A,26.61,N/A,13.327339,N/A,1.0,N/A,NaN,N/A,N/A
56,BAC,27.500,N/A,12.73,N/A,0.896244,N/A,1.0,N/A,NaN,N/A,N/A
64,BK,37.270,N/A,8.27,N/A,0.814026,N/A,1.0,N/A,NaN,N/A,N/A
65,BKNG,1827.800,N/A,30.88,N/A,12.749662,N/A,1.0,N/A,NaN,N/A,N/A
75,C,54.355,N/A,9.42,N/A,0.583137,N/A,1.0,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. Specifically, we will work on the EV/EBITDA and EV/GP columns, since those contain the majority of our missing data.

Here is the code to do this: